This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
def create_url(database_code: str, dataset_code: str, return_format: str):
    url = "https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}"

    return url.format(
        database_code=database_code, dataset_code=dataset_code, return_format=return_format)

database_code = "FSE"
dataset_code = "AFX_X"
return_format = "json"

req_url = create_url(
    database_code=database_code, dataset_code=dataset_code, return_format=return_format)

print(req_url)
r = requests.get(req_url, params={"api_key": API_KEY})
# print(r.url)

if r.status_code == 200:
    print("Success")
    content = r.json()
else:
    print("Failed")
    raise RuntimeError(
        "Something went wrong\n"
        f"Status code: {r.status_code}\n" + r.content.decode())


https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json
Success


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(content["dataset_data"].keys())
print(content["dataset_data"]["column_names"])

# For reference
for index, name in enumerate(content["dataset_data"]["column_names"]):
    print(f"{index}: {name}")

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
0: Date
1: Open
2: High
3: Low
4: Close
5: Change
6: Traded Volume
7: Turnover
8: Last Price of the Day
9: Daily Traded Units
10: Daily Turnover


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# 1 Collect data
database_code = "FSE"
dataset_code = "AFX_X"
return_format = "json"

create_url(database_code, dataset_code, return_format)

params = {
    "api_key": API_KEY,
    "start_date": "2017-01-01",
    "end_date": "2018-01-01"
}

r = requests.get(req_url, params=params)

# 2 convert to dictionary
response = r.json()

In [76]:
# 3 Calculate highest and lowest
cols = response["dataset_data"]["column_names"]
data = response["dataset_data"]["data"]

highs = [x[2] for x in data]
lows = [x[3] for x in data]

highest = max(highs)
lowest = min(lows)
print(f"3.\nHighest is {highest}\nLowest is {lowest}")


# 4 Largest change in a day
deltas = [high - low for high, low in zip(highs, lows)]
print(f"4.\nLargest delta in a day is: {max(deltas)}")


# 5 Largest change between two days based on closing price
closing = [x[4] for x in data]
# Consider positive and negative deltas
import heapq
delta_closing = []
for day_ahead, day_behind in zip(closing[:-1], closing[1:]):
    heapq.heappush(delta_closing, (abs(day_ahead - day_behind), day_ahead - day_behind))
largest_delta = heapq.nlargest(1, delta_closing)
print(f"5.\nLargest change between two consecutive days is: {largest_delta[0][1]}")


# 6 Avg daily trading volume for the year
volumes = [x[6] for x in data]
avg_volume = sum(volumes)/len(volumes)
print(f"6.\nAverage daily trading volume for 2017 is: {avg_volume}")


# 7 Median trading volume
sorted_volumes = sorted(volumes)
print(f"7.\nMedian value is: {sorted_volumes[len(sorted_volumes)//2]}")


3.
Highest is 53.54
Lowest is 33.62
4.
Largest delta in a day is: 2.8100000000000023
5.
Largest change between two consecutive days is: -2.559999999999995
6.
Average daily trading volume for 2017 is: 89124.33725490196
7.
Median value is: 76286.0
